This notebook was adapted from David Shean's tutorial on pulling SNOTEL data (located here: )

In [1]:
# this is the CUAHSI API endpoint
# http://his.cuahsi.org/wofws.html
wsdlurl = 'https://hydroportal.cuahsi.org/Snotel/cuahsi_1_1.asmx?WSDL'

In [2]:
#Install directly from github repo main branch
# #%pip install -q git+https://github.com/ulmo-dev/ulmo.git
import ulmo

In [3]:
# import packages, had some issues importing/installing geopandas and contextily, but got it eventually

import os
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import contextily as ctx


In [4]:
# identify SNOTEL site locations
sites = ulmo.cuahsi.wof.get_sites(wsdlurl)

In [5]:
# preview an item in the dictionary to see what we pulled
next(iter(sites.items()))

('SNOTEL:301_CA_SNTL',
 {'code': '301_CA_SNTL',
  'name': 'Adin Mtn',
  'network': 'SNOTEL',
  'location': {'latitude': '41.2358283996582',
   'longitude': '-120.79192352294922'},
  'elevation_m': '1886.7120361328125',
  'site_property': {'county': 'Modoc',
   'state': 'California',
   'site_comments': 'beginDate=10/1/1983 12:00:00 AM|endDate=1/1/2100 12:00:00 AM|HUC=180200021403|HUD=18020002|TimeZone=-8.0|actonId=20H13S|shefId=ADMC1|stationTriplet=301:CA:SNTL|isActive=True',
   'pos_accuracy_m': '0'}})

In [34]:
# convert to pandas dataframe from dictionary and drop na values
sites_df = pd.DataFrame.from_dict(sites, orient='index').dropna()
sites_df

,code,name,network,location,elevation_m,site_property
SNOTEL:301_CA_SNTL,301_CA_SNTL,Adin Mtn,SNOTEL,"{'latitude': '41.2358283996582', 'longitude': ...",1886.7120361328125,"{'county': 'Modoc', 'state': 'California', 'si..."
SNOTEL:907_UT_SNTL,907_UT_SNTL,Agua Canyon,SNOTEL,"{'latitude': '37.522171020507813', 'longitude'...",2712.719970703125,"{'county': 'Kane', 'state': 'Utah', 'site_comm..."
SNOTEL:916_MT_SNTL,916_MT_SNTL,Albro Lake,SNOTEL,"{'latitude': '45.59722900390625', 'longitude':...",2529.840087890625,"{'county': 'Madison', 'state': 'Montana', 'sit..."
SNOTEL:1267_AK_SNTL,1267_AK_SNTL,Alexander Lake,SNOTEL,"{'latitude': '61.749668121337891', 'longitude'...",48.768001556396484,"{'county': 'Matanuska-Susitna', 'state': 'Alas..."
SNOTEL:908_WA_SNTL,908_WA_SNTL,Alpine Meadows,SNOTEL,"{'latitude': '47.779571533203125', 'longitude'...",1066.800048828125,"{'county': 'King', 'state': 'Washington', 'sit..."
...,...,...,...,...,...,...
SNOTEL:877_AZ_SNTL,877_AZ_SNTL,Workman Creek,SNOTEL,"{'latitude': '33.812419891357422', 'longitude'...",2103.1201171875,"{'county': 'Gila', 'state': 'Arizona', 'site_c..."
SNOTEL:1228_UT_SNTL,1228_UT_SNTL,Wrigley Creek,SNOTEL,"{'latitude': '39.132331848144531', 'longitude'...",2842.86962890625,"{'county': 'Sanpete', 'state': 'Utah', 'site_c..."
SNOTEL:1197_UT_SNTL,1197_UT_SNTL,Yankee Reservoir,SNOTEL,"{'latitude': '37.747970581054688', 'longitude'...",2649.321533203125,"{'county': 'Iron', 'state': 'Utah', 'site_comm..."
SNOTEL:878_WY_SNTL,878_WY_SNTL,Younts Peak,SNOTEL,"{'latitude': '43.9322509765625', 'longitude': ...",2545.080078125,"{'county': 'Park', 'state': 'Wyoming', 'site_c..."


In [35]:
# clean up dataframe and prepare point geometry objects, need x and y locs
# do this by creating a new column (geometry) and pulling lat and long info and use Point package to pull info from dict
sites_df['geometry']=[Point(float(loc['longitude']),float(loc['latitude'])) for loc in sites_df['location']]
sites_df = pd.concat([sites_df, sites_df["location"].apply(pd.Series)], axis=1)
sites_df
sites_df = sites_df.drop(columns='location')
sites_df = sites_df.astype({'elevation_m':float,'latitude':float,'longitude':float})

In [36]:
sites_df

,code,name,network,elevation_m,site_property,geometry,latitude,longitude
SNOTEL:301_CA_SNTL,301_CA_SNTL,Adin Mtn,SNOTEL,1886.712036,"{'county': 'Modoc', 'state': 'California', 'si...",POINT (-120.7919235229492 41.2358283996582),41.235828,-120.791924
SNOTEL:907_UT_SNTL,907_UT_SNTL,Agua Canyon,SNOTEL,2712.719971,"{'county': 'Kane', 'state': 'Utah', 'site_comm...",POINT (-112.2711791992188 37.52217102050781),37.522171,-112.271179
SNOTEL:916_MT_SNTL,916_MT_SNTL,Albro Lake,SNOTEL,2529.840088,"{'county': 'Madison', 'state': 'Montana', 'sit...",POINT (-111.9590225219727 45.59722900390625),45.597229,-111.959023
SNOTEL:1267_AK_SNTL,1267_AK_SNTL,Alexander Lake,SNOTEL,48.768002,"{'county': 'Matanuska-Susitna', 'state': 'Alas...",POINT (-150.8896636962891 61.74966812133789),61.749668,-150.889664
SNOTEL:908_WA_SNTL,908_WA_SNTL,Alpine Meadows,SNOTEL,1066.800049,"{'county': 'King', 'state': 'Washington', 'sit...",POINT (-121.6984710693359 47.77957153320312),47.779572,-121.698471
...,...,...,...,...,...,...,...,...
SNOTEL:877_AZ_SNTL,877_AZ_SNTL,Workman Creek,SNOTEL,2103.120117,"{'county': 'Gila', 'state': 'Arizona', 'site_c...",POINT (-110.9177322387695 33.81241989135742),33.812420,-110.917732
SNOTEL:1228_UT_SNTL,1228_UT_SNTL,Wrigley Creek,SNOTEL,2842.869629,"{'county': 'Sanpete', 'state': 'Utah', 'site_c...",POINT (-111.3568496704102 39.13233184814453),39.132332,-111.356850
SNOTEL:1197_UT_SNTL,1197_UT_SNTL,Yankee Reservoir,SNOTEL,2649.321533,"{'county': 'Iron', 'state': 'Utah', 'site_comm...",POINT (-112.7749481201172 37.74797058105469),37.747971,-112.774948
SNOTEL:878_WY_SNTL,878_WY_SNTL,Younts Peak,SNOTEL,2545.080078,"{'county': 'Park', 'state': 'Wyoming', 'site_c...",POINT (-109.8177490234375 43.9322509765625),43.932251,-109.817749


In [37]:
# gathers all snotel sites
sites_gdf_all = gpd.GeoDataFrame(sites_df, crs='EPSG:4326')
# gathers all snotel sites in Washingto# n
# sites_gme_wa = sites_gdf_all[sites_gdf_all.index.str.cont(nME'WA')
sites_gme_wa = sites_gdf_all[sites_gdf_all.index.str.contains('WA')]

In [38]:
sites_gdf_all

,code,name,network,elevation_m,site_property,geometry,latitude,longitude
SNOTEL:301_CA_SNTL,301_CA_SNTL,Adin Mtn,SNOTEL,1886.712036,"{'county': 'Modoc', 'state': 'California', 'si...",POINT (-120.79192 41.23583),41.235828,-120.791924
SNOTEL:907_UT_SNTL,907_UT_SNTL,Agua Canyon,SNOTEL,2712.719971,"{'county': 'Kane', 'state': 'Utah', 'site_comm...",POINT (-112.27118 37.52217),37.522171,-112.271179
SNOTEL:916_MT_SNTL,916_MT_SNTL,Albro Lake,SNOTEL,2529.840088,"{'county': 'Madison', 'state': 'Montana', 'sit...",POINT (-111.95902 45.59723),45.597229,-111.959023
SNOTEL:1267_AK_SNTL,1267_AK_SNTL,Alexander Lake,SNOTEL,48.768002,"{'county': 'Matanuska-Susitna', 'state': 'Alas...",POINT (-150.88966 61.74967),61.749668,-150.889664
SNOTEL:908_WA_SNTL,908_WA_SNTL,Alpine Meadows,SNOTEL,1066.800049,"{'county': 'King', 'state': 'Washington', 'sit...",POINT (-121.69847 47.77957),47.779572,-121.698471
...,...,...,...,...,...,...,...,...
SNOTEL:877_AZ_SNTL,877_AZ_SNTL,Workman Creek,SNOTEL,2103.120117,"{'county': 'Gila', 'state': 'Arizona', 'site_c...",POINT (-110.91773 33.81242),33.812420,-110.917732
SNOTEL:1228_UT_SNTL,1228_UT_SNTL,Wrigley Creek,SNOTEL,2842.869629,"{'county': 'Sanpete', 'state': 'Utah', 'site_c...",POINT (-111.35685 39.13233),39.132332,-111.356850
SNOTEL:1197_UT_SNTL,1197_UT_SNTL,Yankee Reservoir,SNOTEL,2649.321533,"{'county': 'Iron', 'state': 'Utah', 'site_comm...",POINT (-112.77495 37.74797),37.747971,-112.774948
SNOTEL:878_WY_SNTL,878_WY_SNTL,Younts Peak,SNOTEL,2545.080078,"{'county': 'Park', 'state': 'Wyoming', 'site_c...",POINT (-109.81775 43.93225),43.932251,-109.817749


In [39]:
sites_gme_wa

,code,name,network,elevation_m,site_property,geometry,latitude,longitude
SNOTEL:908_WA_SNTL,908_WA_SNTL,Alpine Meadows,SNOTEL,1066.800049,"{'county': 'King', 'state': 'Washington', 'sit...",POINT (-121.69847 47.77957),47.779572,-121.698471
SNOTEL:990_WA_SNTL,990_WA_SNTL,Beaver Pass,SNOTEL,1106.423950,"{'county': 'Whatcom', 'state': 'Washington', '...",POINT (-121.25550 48.87930),48.879299,-121.255501
SNOTEL:352_WA_SNTL,352_WA_SNTL,Blewett Pass,SNOTEL,1292.352051,"{'county': 'Chelan', 'state': 'Washington', 's...",POINT (-120.67960 47.35037),47.350368,-120.679604
SNOTEL:1080_WA_SNTL,1080_WA_SNTL,Brown Top,SNOTEL,1776.984009,"{'county': 'Whatcom', 'state': 'Washington', '...",POINT (-121.19713 48.92755),48.927551,-121.197128
SNOTEL:1107_WA_SNTL,1107_WA_SNTL,Buckinghorse,SNOTEL,1484.375977,"{'county': 'Jefferson', 'state': 'Washington',...",POINT (-123.45747 47.70860),47.708599,-123.457474
...,...,...,...,...,...,...,...,...
SNOTEL:832_WA_SNTL,832_WA_SNTL,Trough,SNOTEL,1670.303955,"{'county': 'Chelan', 'state': 'Washington', 's...",POINT (-120.29412 47.23328),47.233280,-120.294121
SNOTEL:841_WA_SNTL,841_WA_SNTL,Upper Wheeler,SNOTEL,1319.784058,"{'county': 'Chelan', 'state': 'Washington', 's...",POINT (-120.37015 47.28734),47.287338,-120.370148
SNOTEL:974_WA_SNTL,974_WA_SNTL,Waterhole,SNOTEL,1527.047974,"{'county': 'Clallam', 'state': 'Washington', '...",POINT (-123.42594 47.94485),47.944851,-123.425941
SNOTEL:909_WA_SNTL,909_WA_SNTL,Wells Creek,SNOTEL,1228.343994,"{'county': 'Whatcom', 'state': 'Washington', '...",POINT (-121.78976 48.86610),48.866100,-121.789757


In [12]:
# Coordinates for eastern washington domain
xmin, xmax, ymin, ymax = [-122.5, -117, 47, 49]
sites_gdf_ewa = sites_gme_wa.cx[xmin:xmax, ymin:ymax]
sites_gdf_ewa.geometry.values

<GeometryArray>
Length: 49, dtype: geometry

In [ ]:
## import for interactive visualizations
# import hvplot.pandas
# from geoviews import tile_sources as gvts

In [ ]:
## geojson of state polygons for later visualization
# states_url = 'http://eric.clst.org/assets/wiki/uploads/Stuff/gz_2010_us_040_00_5m.json'
# states_gdf = gpd.read_file(states_url)

In [ ]:
## makes nice figure of snotel locations with esri map 
# ewa_snotel_sites = sites_gdf_ewa
# map_tiles = gvts.EsriImagery
# map_tiles * ewa_snotel_sites.to_crs('EPSG:3857').hvplot(hover_cols=['index','name']) * states_gdf.to_crs('EPSG:3857').hvplot(color='none')

In [14]:
# snow and swe variables
met_vars = ['SNOTEL:SNWD_D','SNOTEL:WTEQ_D']

today = datetime.today().strftime('%Y-%m-%d')
# function will take in a sitecode, variable code, start and end date and return variable data for that range and location as df
def snotel_fetch(sitecode, variablecode, start_date='1950-01-01', end_date=today):
    values_df = None
    try:
        # start by requesting data from the server
        print('Requesting Data from Server...')
        site_values = ulmo.cuahsi.wof.get_values(wsdlurl,sitecode, variablecode, start=start_date, end=end_date)
        # convert to pandas dataframe
        values_df = pd.DataFrame.from_dict(site_values['values'])
        # parse the datetime values to Pandas Timestamp object
        print('Cleaning Data...')
        values_df['datetime'] = pd.to_datetime(values_df['datetime'], utc=False)
        # set df index to datetime
        values_df = values_df.set_index('datetime')
        # convert values to float and replace -9999 with NaN
        values_df['value'] = pd.to_numeric(values_df['value']).replace(-9999,np.nan)
        # rename values column after variable code
        values_df.rename(columns = {'value':variablecode}, inplace = True)
        # remove lower quality records
        values_df = values_df[values_df['quality_control_level_code']=='1']
        print('Done!')
    except:
        print('Unable to fetch %s' % variablecode)
    return values_df

In [18]:
def getStation_output(outpath ,fname, sitecode, start, end, variables=met_vars):
    output = []
    for var in variables:
        if var == 'SNOTEL:SNWD_D':
            name = fname+'depth'
        elif var == 'SNOTEL:WTEQ_D':
            name = fname+'swe'
        print('Fetching {0} for {1}'.format(var, sitecode))
        variable_code = var # 0 for depth, 1 for swe
        site_snotel = snotel_fetch(site_code, variable_code,start_date=start,end_date=end)
        return site_snotel
        # site_snotel.to_excel(os.path.join(outpath,name+'.xlsx'))
        # print('Produced excel located here {0}'.format(os.path.join(outpath,name+'.xlsx')))
        # site_snotel.to_csv(os.path.join(outpath,name+'.csv')) 
        # print('Produced excel located here {0}'.format(os.path.join(outpath,name+'.csv')))
        # output.append(site_snotel)
    return tuple(output)


In [20]:
site_code = 'SNOTEL:1259_WA_SNTL' # for muckamuck station
start, end = '2018-10-01', '2019-09-30' # change dates if desired
outpath = '/glade/scratch/rossamower/snow/snowmodel/washington/python/snotel'
fname = 'Muckamuck_SNOTEL_2018_'
muckamuck_depth = getStation_output(outpath, fname, site_code, start, end)
# muckamuck_depth, muckamuck_swe = getStation_output(outpath, fname, site_code, start, end)

Fetching SNOTEL:SNWD_D for SNOTEL:1259_WA_SNTL
Requesting Data from Server...
Cleaning Data...
Done!


In [24]:
muckamuck_depth[muckamuck_depth.index == '2019-02-01']

,SNOTEL:SNWD_D,qualifiers,censor_code,date_time_utc,method_id,method_code,source_code,quality_control_level_code
datetime,,,,,,,,
2019-02-01,19,V,nc,2019-02-01T00:00:00,0,0,1,1
